# Test analysis

Statistical analysis for 10 repetitions of a 5 minute test and for 10 repetitions of a 1 minute test of the **Robot-Shop** application

### Workloads

- **5 minute test**:
    - simulated day length: 5 minutes = 300 seconds
    - 20 requests per second
    - about 6000 total requests

- **1 minute test** (compressed test):
    - simulated day length: 1 minutes = 60 seconds
    - 100 requests per second
    - about 6000 total requests

### Imports and functions

In [1]:
import os
import pathlib
import sys

import numpy as np
import pandas as pd
from IPython.display import HTML, display, display_html

mod_path = os.path.abspath(os.path.join("../../src/alyslib"))
if mod_path not in sys.path:
    sys.path.append(mod_path)

import alyslib

In [2]:
# function that returns a list that contains
# the mean of the TimeDeltas for every dataframe
def get_means(df):
    l = []
    for d in df:
        l.append(d.TimeDelta.mean())
    return l


# function that calculates the confidence interval
# (with `z_score`=1.96, returns a 95% confidence interval)
def conf_interval(data, z_score=1.96):
    mean = np.mean(data)
    std = np.std(data)
    size = len(data)
    err = z_score * (std / np.sqrt(size))
    return (mean - err, mean + err)

### Datasets - import

In [3]:
l = alyslib.import_data("./data", "net.gen")

### DataFrames - building

In [4]:
d0, d1 = alyslib.build_dfs(l)

In [5]:
dfmerge = d0 + d1

### DataFrames - cleaning network noise

For the analysis of the tests we cannot have **SendIP** and **RecvIP** differences. We clean the network noise for every pair of tests.

In [6]:
alyslib.clean_network_noise(dfmerge)

dataframe 139630949485104, removed 140 items, [2262, 2263, 2265, 2267, 2269]
dataframe 139630949485104, removed 120 items, [2264, 2266, 2268, 2271, 2272]
dataframe 139630949455456, removed 140 items, [3537, 3538, 3540, 3542, 3544]
dataframe 139630949455456, removed 121 items, [3539, 3541, 3543, 3546, 3547]
dataframe 139630949491584, removed 3 items, [13479, 13476, 13477]
dataframe 139630949491584, removed 1 items, [13478]
dataframe 139630949452864, removed 3 items, [3307, 3304, 3306]
dataframe 139630949452864, removed 1 items, [3305]
dataframe 139630949496912, removed 3 items, [16848, 16847, 16849]
dataframe 139630949496912, removed 1 items, [16850]
dataframe 139630949485104, removed 3 items, [24477, 24474, 24476]
dataframe 139630949485104, removed 1 items, [24475]
dataframe 139630949494656, removed 3 items, [8573, 8572, 8574]
dataframe 139630949494656, removed 1 items, [8575]
dataframe 139630949492448, removed 3 items, [4678, 4677, 4679]
dataframe 139630949492448, removed 1 items, [46

### Dataframes - sorting by Timestamp

In [7]:
alyslib.sort_by_key(dfmerge, "Timestamp")

### Dataframes - generating column Elapsed time

In [8]:
alyslib.cmp_elapsed(dfmerge)

### DataFrames - removing TimeDelta outliers

We treat as outliers every TimeDelta that is greater than the third TimeDelta quantile (q3 = 0.75).

In [9]:
alyslib.clean_timedelta_outliers(dfmerge)

### Analysis - 5 minutes test

In [10]:
d0m = get_means(d0)

In [11]:
display(pd.DataFrame(d0m, columns=["means"]))
display(
    pd.DataFrame(
        [[np.mean(d0m), np.std(d0m), conf_interval(d0m)]],
        columns=["mean of means", "std of means", "95% conf interval"],
    )
)

,means
0,0.008978
1,0.009572
2,0.008433
3,0.010115
4,0.008902
5,0.009453
6,0.009996
7,0.008889
8,0.008959
9,0.010145


,mean of means,std of means,95% conf interval
0,0.009344,0.000569,"(0.008991441518557587, 0.009696982279394634)"


1. we calculated the **TimeDelta mean** for every DataFrame generated for the current test (we have 10 repetitions, so we have 10 DataFrames).
2. we calculated the **mean of the means** calculated above
3. we calculated the **standard deviation of the means** calculated above
4. we calculated the **95% confidence interval of the means** calculated above

### Analysis - 1 minute test

In [12]:
d1m = get_means(d1)

In [13]:
display(pd.DataFrame(d1m, columns=["means"]))
display(
    pd.DataFrame(
        [[np.mean(d1m), np.std(d1m), conf_interval(d1m)]],
        columns=["mean of means", "std of means", "95% conf interval"],
    )
)

,means
0,0.001864
1,0.002086
2,0.001847
3,0.001849
4,0.002050
5,0.001858
6,0.001865
7,0.001850
8,0.001871
9,0.001895


,mean of means,std of means,95% conf interval
0,0.001904,0.000084,"(0.001851678829434816, 0.001955344797007489)"


We performed the same calculations as the previous test with this new test (again we have 10 repetitions, so we have 10 DataFrames for the current test)

### Analysis - comparison between the tests

In [14]:
display(
    pd.DataFrame(
        [
            [np.mean(d0m), np.std(d0m), conf_interval(d0m)],
            [np.mean(d1m), np.std(d1m), conf_interval(d1m)],
        ],
        columns=["mean of means", "std of means", "95% conf interval"],
        index=["5m test", "1m test"],
    )
)

,mean of means,std of means,95% conf interval
5m test,0.009344,0.000569,"(0.008991441518557587, 0.009696982279394634)"
1m test,0.001904,0.000084,"(0.001851678829434816, 0.001955344797007489)"


In conclusion, we present above the achieved results to compare the results of both tests simultaneously